# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,25.86,72,100,2.51,PF,1675466639
1,1,flinders,-34.5833,150.8552,20.77,52,0,3.58,AU,1675466039
2,2,ponta do sol,32.6667,-17.1000,15.56,51,40,2.62,PT,1675466639
3,3,saskylakh,71.9167,114.0833,-42.04,100,74,1.74,RU,1675466639
4,4,new norfolk,-42.7826,147.0587,14.95,88,100,1.34,AU,1675466640


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
# %%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",frame_width = 700,
    frame_height = 500,
    size = "Population",
    scale = 0.01,
    color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
narrowed_city_df=narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
129,129,calvinia,-31.4707,19.7760,22.15,32,0,1.36,ZA,1675466729
141,141,cabo san lucas,22.8909,-109.9124,25.17,33,0,3.09,MX,1675466334
163,163,port elizabeth,-33.9180,25.5701,22.28,91,0,3.60,ZA,1675466745
323,323,coihaique,-45.5752,-72.0662,26.97,29,0,3.60,CL,1675466823
436,436,kiama,-34.6833,150.8667,21.99,48,0,2.24,AU,1675466884
563,563,nelson bay,-32.7167,152.1500,23.94,40,0,4.47,AU,1675466924


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
129,calvinia,ZA,-31.4707,19.7760,32,
141,cabo san lucas,MX,22.8909,-109.9124,33,
163,port elizabeth,ZA,-33.9180,25.5701,91,
323,coihaique,CL,-45.5752,-72.0662,29,
436,kiama,AU,-34.6833,150.8667,48,
563,nelson bay,AU,-32.7167,152.1500,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
calvinia - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
port elizabeth - nearest hotel: Waterford Hotel
coihaique - nearest hotel: No hotel found
kiama - nearest hotel: Nova Kiama
nelson bay - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
129,calvinia,ZA,-31.4707,19.7760,32,No hotel found
141,cabo san lucas,MX,22.8909,-109.9124,33,Comfort Rooms
163,port elizabeth,ZA,-33.9180,25.5701,91,Waterford Hotel
323,coihaique,CL,-45.5752,-72.0662,29,No hotel found
436,kiama,AU,-34.6833,150.8667,48,Nova Kiama
563,nelson bay,AU,-32.7167,152.1500,40,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)